In [67]:
import pandas as pd

demographic_df = pd.read_pickle('../data/social_explorer/demographic_df.pkl')
demographic_df.drop('Qualifying Name', axis=1, inplace=True)
demographic_df = demographic_df.dropna()
demographic_df = demographic_df.reset_index(drop=True)

environment_df = pd.read_pickle('../data/social_explorer/environment_df.pkl')
environment_df = environment_df.dropna()
environment_df = environment_df.reset_index(drop=True)

housing_df = pd.read_pickle('../data/social_explorer/housing_df.pkl')
housing_df.drop('Qualifying Name', axis=1, inplace=True)
housing_df = housing_df.dropna()
housing_df = housing_df.reset_index(drop=True)

crime_df = pd.read_pickle('../data/social_explorer/crime_df.pkl')
crime_df.drop('Qualifying Name', axis=1, inplace=True)
crime_df = crime_df.dropna()
crime_df = crime_df.reset_index(drop=True)


merged_df = pd.merge(environment_df, demographic_df, on='FIPS', how='left')
# merged_df = pd.merge(merged_df, housing_df, on='FIPS', how='left')
merged_df = pd.merge(merged_df, crime_df, on='FIPS', how='left')
merged_df.columns

Index(['FIPS', 'Qualifying Name', 'Total Area', 'Open Water %', 'Ice/Snow %',
       'Developed Open Space %', 'Developed Low Intensity %',
       'Developed Medium Intensity %', 'Developed High Intensity %',
       'Barren Land %', 'Deciduous Forest %', 'Evergreen Forest %',
       'Mixed Forest %', 'Shrub/Scrub %', 'Grassland/Herbaceous %',
       'Pasture/Hay %', 'Cultivated Crops %', 'Woody Wetlands %',
       'Emergent Herbaceous Wetlands %', '2020 Total Population',
       '2020 Population Density', 'White 2020 %',
       'African American or Black 2020 %',
       'American Indian and Alaska Native 2020 %', 'Asian 2020 %',
       'Native Hawaiian and Other Pacific Islander 2020 %', 'Other 2020 %',
       'Two or More Races 2020 %', 'Hispanic 2020 %',
       'Total Violent and Property Crimes %'],
      dtype='object')

In [78]:
import pandas as pd
import geopandas as gpd
import folium
from folium import LinearColormap

df = merged_df
# Convert FIPS to string for merging purposes
df['FIPS'] = df['FIPS'].astype(str)
# df = df[['FIPS','Total Violent and Property Crimes %']]

# Load county shapefile
county_shapefile_path = '../mapVis/county_shape_files/cb_2018_us_county_500k.shp'
county_geo_data = gpd.read_file(county_shapefile_path)
# county_geo_data['FIPS'] = county_geo_data['FIPS'].astype(str)
# county_geo_data.head

# Ensure the FIPS column is present and converted to string
if 'GEOID' in county_geo_data.columns:
    county_geo_data['FIPS'] = county_geo_data['GEOID'].astype(str)
elif 'FIPS' in county_geo_data.columns:
    county_geo_data['FIPS'] = county_geo_data['FIPS'].astype(str)
else:
    raise ValueError("FIPS column not found in county shapefile")

# Merge county shapefile data with crime data
merged_county_data = county_geo_data.merge(df, on='FIPS')
# merged_county_data.head()

# Load state shapefile
state_shapefile_path = '../mapVis/state_shape_files/cb_2018_us_state_500k.shp'
state_geo_data = gpd.read_file(state_shapefile_path)


def gen_vis(hm_category):
    # Determine the min and max values for crime rates
    min_value = merged_county_data[hm_category].min()
    max_value = merged_county_data[hm_category].max()
    
    # Create a custom color scale
    color_scale = LinearColormap(
        colors=['green', 'yellow', 'red'],
        index=[min_value, (min_value + max_value) / 2, max_value],
        caption='Total Violent and Property Crimes %'
    )
    
    # Create a folium map centered at a specified location
    m = folium.Map(location=[32.806671, -86.791130], zoom_start=7)
    
    # Add state boundaries with black lines and no fill
    folium.GeoJson(
        state_geo_data,
        name='State Boundaries',
        style_function=lambda feature: {
            'color': 'black',
            'weight': 3,
            'fillOpacity': 0  # No fill
        }
    ).add_to(m)
    
    # Add county boundaries with color based on param field
    def style_function(feature):
        # Retrieve the FIPS code and crime rate from the feature properties
        fips = feature['properties']['FIPS']
        crime_rate = merged_county_data.loc[merged_county_data['FIPS'] == fips, hm_category].values[0]
    
        # Apply color scale based on the crime rate
        color = color_scale(crime_rate)
        return {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        }
    
    # Create a GeoJson layer for counties with custom style and tooltips
    folium.GeoJson(
        merged_county_data.__geo_interface__,
        name='County Boundaries',
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=['NAME', 'Total Area', 'Open Water %', 'Ice/Snow %',
           'Developed Open Space %', 'Developed Low Intensity %',
           'Developed Medium Intensity %', 'Developed High Intensity %',
           'Barren Land %', 'Deciduous Forest %', 'Evergreen Forest %',
           'Mixed Forest %', 'Shrub/Scrub %', 'Grassland/Herbaceous %',
           'Pasture/Hay %', 'Cultivated Crops %', 'Woody Wetlands %',
           'Emergent Herbaceous Wetlands %', '2020 Total Population',
           '2020 Population Density', 'White 2020 %',
           'African American or Black 2020 %',
           'American Indian and Alaska Native 2020 %', 'Asian 2020 %',
           'Native Hawaiian and Other Pacific Islander 2020 %', 'Other 2020 %',
           'Two or More Races 2020 %', 'Hispanic 2020 %',
           'Total Violent and Property Crimes %'],
            aliases=['County:', 'Total Area', 'Open Water %', 'Ice/Snow %',
           'Developed Open Space %', 'Developed Low Intensity %',
           'Developed Medium Intensity %', 'Developed High Intensity %',
           'Barren Land %', 'Deciduous Forest %', 'Evergreen Forest %',
           'Mixed Forest %', 'Shrub/Scrub %', 'Grassland/Herbaceous %',
           'Pasture/Hay %', 'Cultivated Crops %', 'Woody Wetlands %',
           'Emergent Herbaceous Wetlands %', '2020 Total Population',
           '2020 Population Density', 'White 2020 %',
           'African American or Black 2020 %',
           'American Indian and Alaska Native 2020 %', 'Asian 2020 %',
           'Native Hawaiian and Other Pacific Islander 2020 %', 'Other 2020 %',
           'Two or More Races 2020 %', 'Hispanic 2020 %',
           'Total Violent and Property Crimes %'],
            localize=True
        )
    ).add_to(m)
    
    # Add the color scale as a legend
    color_scale.add_to(m)
    
    # Add layer control to the map
    folium.LayerControl().add_to(m)
    
    # Save the map as an HTML file
    m.save('us_state_and_county_boundaries.html')

# hm_category = 'Total Violent and Property Crimes %'
# hm_category = 'White 2020 %'
hm_category = 'Asian 2020 %'
gen_vis(hm_category)